In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# Question 1

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

In [4]:
spark.version

'3.5.0'

In [5]:
#Question 2

In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('D:/DE-ZoomCamp/week_5_batch/code/fhv_tripdata_2019-10.csv.gz')

In [7]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [8]:
df.filter(df["SR_Flag"].isNotNull()).show()

+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|dispatching_base_num|pickup_datetime|dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+
+--------------------+---------------+----------------+------------+------------+-------+----------------------+



In [9]:
from pyspark.sql import types

In [10]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),  
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
    ])

In [11]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [22]:
from pyspark.sql.functions import month, year

In [23]:
# df_october_2019 = df.filter(
#     (year(df["pickup_datetime"]) == 2019) & 
#     (month(df["pickup_datetime"]) == 10) & 
#     (year(df["dropOff_datetime"]) == 2019) & 
#     (month(df["dropOff_datetime"]) == 10)
# )

In [25]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [26]:
df = df.repartition(6)

In [27]:
df.write.parquet('fhv/2019/10/', mode='overwrite')

In [28]:
!du -h fhv

39M	fhv/2019/10
39M	fhv/2019
39M	fhv


In [29]:
!ls -lh fhv/2019/10

total 39M
-rw-r--r-- 1 Armut 197121    0 Feb 26 17:21 _SUCCESS
-rw-r--r-- 1 Armut 197121 6.4M Feb 26 17:21 part-00000-41b48930-19ea-4fbf-9b63-76ec77c36f91-c000.snappy.parquet
-rw-r--r-- 1 Armut 197121 6.4M Feb 26 17:21 part-00001-41b48930-19ea-4fbf-9b63-76ec77c36f91-c000.snappy.parquet
-rw-r--r-- 1 Armut 197121 6.4M Feb 26 17:21 part-00002-41b48930-19ea-4fbf-9b63-76ec77c36f91-c000.snappy.parquet
-rw-r--r-- 1 Armut 197121 6.4M Feb 26 17:21 part-00003-41b48930-19ea-4fbf-9b63-76ec77c36f91-c000.snappy.parquet
-rw-r--r-- 1 Armut 197121 6.4M Feb 26 17:21 part-00004-41b48930-19ea-4fbf-9b63-76ec77c36f91-c000.snappy.parquet
-rw-r--r-- 1 Armut 197121 6.4M Feb 26 17:21 part-00005-41b48930-19ea-4fbf-9b63-76ec77c36f91-c000.snappy.parquet


In [30]:
# QUESTION 3

In [31]:
from pyspark.sql import functions as F

In [32]:
df = spark.read.parquet('fhv/2019/10/*')

In [33]:
df.registerTempTable('fhv')

C:\Users\Armut\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [34]:
spark.sql("""
SELECT
    COUNT(*)
FROM
    fhv
WHERE
    DATE(pickup_datetime) = '2019-10-15'
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [35]:
# Question 4

In [36]:
from pyspark.sql.functions import max, desc

In [67]:
spark.sql("""
SELECT
    MAX((unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime)) / 3600) AS duration
FROM
    fhv
""").show()


+--------+
|duration|
+--------+
|631152.5|
+--------+



In [68]:
# Question 5

In [89]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [90]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [91]:
df_zone = df_zone \
    .withColumnRenamed('LocationID', 'ZoneLocationID') \
    .withColumnRenamed('Borough', 'ZoneBorough') \
    .drop('service_zone')

In [92]:
df_zone.show()

+--------------+-------------+--------------------+
|ZoneLocationID|  ZoneBorough|                Zone|
+--------------+-------------+--------------------+
|             1|          EWR|      Newark Airport|
|             2|       Queens|         Jamaica Bay|
|             3|        Bronx|Allerton/Pelham G...|
|             4|    Manhattan|       Alphabet City|
|             5|Staten Island|       Arden Heights|
|             6|Staten Island|Arrochar/Fort Wad...|
|             7|       Queens|             Astoria|
|             8|       Queens|        Astoria Park|
|             9|       Queens|          Auburndale|
|            10|       Queens|        Baisley Park|
|            11|     Brooklyn|          Bath Beach|
|            12|    Manhattan|        Battery Park|
|            13|    Manhattan|   Battery Park City|
|            14|     Brooklyn|           Bay Ridge|
|            15|       Queens|Bay Terrace/Fort ...|
|            16|       Queens|             Bayside|
|           

In [96]:
df_zone.createOrReplaceTempView("df_zone")
df.createOrReplaceTempView("df")

In [107]:
from pyspark.sql.functions import min

In [112]:
spark.sql("""
SELECT 
    df_zone.Zone,
    COUNT(df_zone.ZoneLocationID)
FROM 
    df
INNER JOIN df_zone ON 
    df.PUlocationID = df_zone.ZoneLocationID
GROUP BY
    1
ORDER BY
    2 ASC
LIMIT 
    1;
""").show()

+-----------+---------------------+
|       Zone|count(ZoneLocationID)|
+-----------+---------------------+
|Jamaica Bay|                    1|
+-----------+---------------------+

